In [33]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from sqlalchemy.orm import Session
from src.sql.models import WikiArticle
from src.sql import engine
from nltk.tokenize import word_tokenize, sent_tokenize
from typing import List, Tuple, Dict
import numpy as np
import re
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support
import pickle
import os

In [34]:
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [35]:
def clean_str(string):
    # remove punctuation
    string = re.sub(r"[^A-Za-z]", " ", string)
    # remove extra spaces
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [36]:
def load_glove_data():
    p = "data/glove.6B/glove.6B.50d.txt"
    word2idx: Dict[str, int] = {}
    idx2word: Dict[int, str] = {}
    embs: List[List[float]] = []
    with open(p, "r", encoding="utf-8") as file:
        i = 0
        for l in file:
            l = l.split()
            word, emb = l[0], l[1:]
            emb = [float(x) for x in emb]
            word2idx[word] = i
            idx2word[i] = word
            embs.append(emb)
            i += 1
        
    
    return word2idx, idx2word, np.array(embs)

In [37]:
def get_train_test_ids(test_portion_size: float = 0.1):
    with Session(engine) as session:
        _ids = []
        items = session.query(WikiArticle.id).limit(10_000).all()
        for item in items:
            _id = item[0]
            _ids.append((_id, "standard"))
            _ids.append((_id, "simple"))
    train_size = 1 - test_portion_size
    train = _ids[:int(len(_ids) * train_size)]
    test = _ids[int(len(_ids) * train_size):]
    return train, test

In [38]:
def build_vocabulary(train_ids: List[Tuple[int, str]], glove_word2idx:Dict[str,int],  glove_idx2word: Dict[int, str]):
    pickle_path = "data/classifier_data/vocabulary.pkl"
    
    if os.path.exists(pickle_path):
        with open(pickle_path, "rb") as file:
            payload = pickle.load(file)
            return payload["word2idx"], payload["idx2word"], payload["unk_idx"], payload["pad_idx"], payload["max_tokens"]
    
    word2idx: Dict[str, int] = {}
    idx2word = {}
    
    for idx, word in glove_idx2word.items():
        word2idx[word] = idx
        idx2word[idx] = word
    
    voc = set()
    max_tokens = 0
    with Session(engine) as session:
        for _id, level in tqdm(train_ids):
            doc = session.query(WikiArticle).filter_by(id=_id).first()
            text = doc.simple_text if level == "simple" else doc.standard_text
            text = clean_str(text)
            tokens = word_tokenize(text)
            for token in tokens:
                voc.add(token)
            max_tokens = max(max_tokens, len(tokens))
    
    unprocessed = []        
    for word in voc:
        glove_idx = glove_word2idx.get(word)
        if glove_idx is None:
            unprocessed.append(word)
    
    i = len(word2idx)
    for word in unprocessed:
        while idx2word.get(i) is not None:
            i += 1
        word2idx[word] = i
        idx2word[i] = word
        i += 1
    
    unk_idx = len(word2idx)
    while idx2word.get(unk_idx) is not None:
        unk_idx += 1
    unk = "<UNK>"
    word2idx[unk] = unk_idx
    idx2word[unk_idx] = unk
    
    pad_idx = len(word2idx)
    while idx2word.get(pad_idx) is not None:
        pad_idx += 1
    pad = "<PAD>"
    word2idx[pad] = pad_idx
    idx2word[pad_idx] = pad
    
    pickle_payload = {
        "word2idx": word2idx,
        "idx2word": idx2word,
        "unk_idx": unk_idx,
        "pad_idx": pad_idx,
        "max_tokens": max_tokens
    }
    
    with open(pickle_path, "wb") as file:
        pickle.dump(pickle_payload, file)
    
    return word2idx, idx2word, unk_idx, pad_idx, max_tokens



In [39]:
def random_vector(size:int) -> torch.Tensor:
    return torch.rand(size, device=device, dtype=torch.float32)

def get_embedding_weights(embs: np.ndarray, idx2word: Dict[int, str]):
    embedding_dim = 50
    vocab_size = len(idx2word)
    weight_matrix = torch.zeros((vocab_size, embedding_dim))
    for idx in idx2word:
        if idx < len(embs):
            weight_matrix[idx] = torch.tensor(embs[idx], device=device, dtype=torch.float32)
        else:
            weight_matrix[idx] = random_vector(embedding_dim)
    return weight_matrix

In [40]:
def encode_text(
    string, 
    word2idx: Dict[str, int], 
    unk_idx: int, 
    limit_sents: int = None
):
    sents = sent_tokenize(string, language="english")
    if limit_sents:
        sents = sents[:limit_sents]
    sents = ".".join(sents)
    text = clean_str(sents)
    toks = word_tokenize(text, language="english")
    tok_idx = [word2idx.get(tok, unk_idx) for tok in toks]
    return tok_idx
    

In [41]:
class WikiArticleDataset(Dataset):
    def __init__(
        self, ids: List[Tuple[int, str]], 
        word2idx: Dict[str, int], 
        unk_tok_idx: int, 
        pad_token_idx: int,
        pad_to_size: int = 1000,
        sent_limit: int = None
    ) -> None:
        super().__init__()
        self._ids = ids
        self.word2idx = word2idx
        self.unk_idx = unk_tok_idx
        self.pad_size = pad_to_size
        self.pad_idx = pad_token_idx
        self.sent_limit = sent_limit
    
    def __len__(self):
        return len(self._ids)

    def __getitem__(self, index) -> Tuple[str, str]:
        _id, level = self._ids[index]
        with Session(engine) as session:
            item = session.query(WikiArticle).filter_by(id=_id).first()
            if level == "standard":
                text = item.standard_text
            else:
                text = item.simple_text
        tok_idx = encode_text(
            text, 
            self.word2idx, 
            self.unk_idx, 
            limit_sents=self.sent_limit
        )
        tok_idx = torch.tensor(tok_idx, device=device, dtype=torch.long)
        
        if len(tok_idx) < self.pad_size:
            pad = torch.tensor([self.pad_idx] * (self.pad_size - len(tok_idx)), device=device, dtype=torch.long)
            tok_idx = torch.cat([tok_idx, pad])
        elif len(tok_idx) > self.pad_size:
            tok_idx = tok_idx[:self.pad_size]
        
        level = torch.tensor([1], device=device) if level == "standard" else torch.tensor([0], device=device)
        return tok_idx, level

In [42]:
class Classifier(nn.Module):
    def __init__(
        self,
        hidden_size: int = 64,
        glove_embs: np.ndarray = None,
        word2idx: Dict[str, int] = None,
        idx2word: Dict[int, str] = None,
        load_emb_weight: bool = True
    ) -> None:
        super().__init__()
        self.glove_embs = glove_embs
        self.word2idx = word2idx
        self.idx2word = idx2word
        
        self.emb_size = 50
        self.embedding = nn.Embedding(len(idx2word), self.emb_size)
        if load_emb_weight:
            self._load_embedding_weight()
        
        self.lstm = nn.LSTM(
            input_size=self.emb_size,
            hidden_size=hidden_size,
            num_layers=3,
            batch_first=True,
        )
        
        self.dropout = nn.Dropout(0.1)
        self.hidden_to_output = nn.Linear(hidden_size, 2)
        self.softmax = nn.LogSoftmax(dim=1)
        
        
    def _load_embedding_weight(self):
        weight = get_embedding_weights(self.glove_embs, self.idx2word)
        self.embedding.load_state_dict({"weight": weight})
        self.embedding.weight.requires_grad = False
    
    def forward(self, document_batch: torch.Tensor):
        document_batch = self.embedding(document_batch)
        lstm_out, _ = self.lstm(document_batch)
        lstm_out = lstm_out[:, -1, :]
        # lstm_out = self.dropout(lstm_out)
        out = self.hidden_to_output(lstm_out)
        out = self.softmax(out)
        return out

In [43]:
def train(
    model: Classifier, 
    train_loader: DataLoader, 
    optimizer: torch.optim.Optimizer, 
    criterion: nn.Module,
    epochs: int = 10,
):  
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1} of {epochs}")
        pbar = tqdm(train_loader)
        for (document_batch, level_batch) in pbar:
            optimizer.zero_grad()
            out = model(document_batch)
            # combine losses
            loss = 0
            for i in range(out.shape[0]):
                prediction = out[i].unsqueeze(0)
                correct = level_batch[i]
                loss += criterion(prediction, correct)
            pbar.set_description(f"Loss: {loss.item()}")
            loss.backward()
            
            optimizer.step()

In [44]:
def evaluate(model: Classifier, test_dataloader: DataLoader):
    y_true = []
    y_pred = []
    for document_batch, level_batch in test_dataloader:
        out = model(document_batch)
        y_true.extend(level_batch.reshape(-1).tolist())
        out = out.argmax(dim=1).reshape(-1).tolist()
        y_pred.extend(out)
    precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred, average="macro")
    print(f"Precision:\t{precision}\nRecall:\t{recall}\nF1\t{fscore}")

In [45]:
train_ids, test_ids = get_train_test_ids()
glove_word2idx, glove_idx2word, glove_embs = load_glove_data()
vocab_word2idx, vocab_idx2word, unk_idx, pad_idx, max_tokens = build_vocabulary(
    train_ids, glove_word2idx, glove_idx2word)



In [47]:

train_dataset = WikiArticleDataset(
    train_ids, 
    vocab_word2idx, 
    unk_idx, pad_idx, 
    pad_to_size=max_tokens, 
    sent_limit=5
)
test_dataset = WikiArticleDataset(
    test_ids, 
    vocab_word2idx, 
    unk_idx, pad_idx, 
    pad_to_size=max_tokens, 
    sent_limit=1
)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [48]:
model_path = "data/classifier_data/model.pt"

if os.path.exists(model_path):
    model = Classifier(
        glove_embs=glove_embs, 
        word2idx=vocab_word2idx, 
        idx2word=vocab_idx2word,
        load_emb_weight=False
    )
    model.load_state_dict(torch.load(model_path))
    model.to(device)

else:
    model = Classifier(
        glove_embs=glove_embs, 
        word2idx=vocab_word2idx, 
        idx2word=vocab_idx2word,
        load_emb_weight=True
    )
    model.to(device)
    optimizer = Adam(model.parameters(), lr=0.001)
    criterion = nn.NLLLoss()
    train(model, train_dataloader, optimizer, criterion, epochs=10)
    torch.save(model.state_dict(), model_path)

Epoch 1 of 10


  0%|          | 0/282 [00:00<?, ?it/s]

: 

: 

In [ ]:
evaluate(model, test_dataloader)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Precision:	0.25
Recall:	0.5
F1	0.3333333333333333


/Users/aron/Library/Caches/pypoetry/virtualenvs/term-paper-nfLWpm6z-py3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
